In [1]:
# Cell 1: Setup, Config, Client Init

import os
import logging
import sys
from decimal import Decimal, ROUND_DOWN, ROUND_UP, InvalidOperation
import time
import math
import pandas as pd
from binance.client import Client
from binance.exceptions import BinanceAPIException, BinanceOrderException, BinanceRequestException
from dotenv import load_dotenv

# --- Configuration ---
QUOTE_ASSET = 'USDT'
MIN_VALUE_TO_SELL_USDT = Decimal('1.00') # Ignore dust below this USDT value
# Pricing Strategy: 'BID' (match best bid), 'BELOW_ASK' (1 tick below best ask), 'MID'
PRICING_STRATEGY = 'BELOW_ASK'
ORDER_BOOK_DEPTH = 5
INTER_ORDER_DELAY_SECONDS = 0.2 # Delay between placing orders

# --- Logging ---
# Configure logging (can copy from other notebooks)
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
log_handler = logging.StreamHandler(sys.stdout)
log_handler.setFormatter(log_formatter)
logger = logging.getLogger()
logger.handlers.clear() # Remove existing handlers
logger.addHandler(log_handler)
logger.setLevel(logging.INFO)

# --- Initialize Binance Client ---
load_dotenv()
API_KEY = os.environ.get('BINANCE_API_KEY')
SECRET_KEY = os.environ.get('BINANCE_SECRET_KEY') or os.environ.get('BINANCE_API_SECRET')
client = None
logging.info("--- Initializing Binance Client for Flatten Utility ---")
if API_KEY and SECRET_KEY:
    try:
        client = Client(API_KEY, SECRET_KEY, tld='us')
        client.ping()
        logging.info("✅ Binance client initialized successfully.")
    except Exception as e:
        logging.error(f"❌ Error initializing Binance client: {e}")
        raise RuntimeError("Client initialization failed.")
else:
    raise ValueError("❌ API Key/Secret not found in environment variables.")


# --- Helper Functions ---
def adjust_price_up(price, tick_size):
    if tick_size <= 0: return price
    ticks = price / tick_size; adjusted_ticks = ticks.to_integral_value(rounding=ROUND_UP); return adjusted_ticks * tick_size
def adjust_price(price, tick_size): # Adjust Down
    if tick_size <= 0: return price; return (price // tick_size) * tick_size
def adjust_qty(quantity, min_q, step_size):
    if step_size <= 0: return quantity
    if quantity < min_q: return Decimal('0')
    num_steps = math.floor((quantity - min_q) / step_size); adj_qty = min_q + (num_steps * step_size); return adj_qty
def format_decimal_for_api(value: Decimal, step_or_tick_size: Decimal) -> str:
    if not isinstance(value, Decimal) or not isinstance(step_or_tick_size, Decimal) or step_or_tick_size <= 0: return f"{value:.8f}" # Fallback
    decimal_places = abs(step_or_tick_size.normalize().as_tuple().exponent); return f"{value:.{decimal_places}f}"

logging.info("Setup Complete. Ready for next step.")
# End of Cell 1

2025-04-10 01:33:17,242 - INFO - --- Initializing Binance Client for Flatten Utility ---
2025-04-10 01:33:17,557 - INFO - ✅ Binance client initialized successfully.
2025-04-10 01:33:17,558 - INFO - Setup Complete. Ready for next step.


In [2]:
# Cell 2: Fetch Balances & Identify Assets to Flatten

import pandas as pd
from decimal import Decimal
import logging

# --- Prerequisites ---
if 'client' not in locals() or client is None: raise RuntimeError("Client not initialized.")
if 'QUOTE_ASSET' not in locals(): raise RuntimeError("QUOTE_ASSET not defined.")
if 'MIN_VALUE_TO_SELL_USDT' not in locals(): raise RuntimeError("MIN_VALUE_TO_SELL_USDT not defined.")

logging.info("--- Fetching Balances & Identifying Assets to Flatten ---")

# --- Fetch Balances ---
portfolio_state = {"balances": pd.DataFrame()} # Initialize
try:
    account_info = client.get_account()
    balances_raw = account_info.get('balances', [])
    processed = []
    for item in balances_raw:
        free = Decimal(item['free']); locked = Decimal(item['locked'])
        if free > 0 or locked > 0: # Include assets even if only locked (though we only sell free)
             processed.append({'Asset': item['asset'], 'Free': free, 'Locked': locked})
    if processed:
         portfolio_state["balances"] = pd.DataFrame(processed).set_index('Asset')
    logging.info(f"Fetched {len(portfolio_state['balances'])} non-zero balance entries.")
except Exception as e:
    logging.error(f"Failed to fetch balances: {e}")
    raise SystemExit("Cannot proceed without balances.")

# --- Fetch Tickers for Valuation ---
all_tickers = {}
try:
    logging.info("Fetching all ticker prices...")
    all_tickers = {t['symbol']: t for t in client.get_ticker()}
    logging.info(f"Fetched {len(all_tickers)} tickers.")
except Exception as e:
     logging.error(f"Failed to fetch tickers: {e}")
     # Can potentially continue but valuation might fail for some assets

# --- Identify Assets ---
balances_df = portfolio_state.get('balances', pd.DataFrame()).copy()
assets_to_flatten = []
total_portfolio_value_approx = Decimal('0')

# Add USDT value
if QUOTE_ASSET in balances_df.index:
     usdt_bal = balances_df.loc[QUOTE_ASSET, 'Free'] + balances_df.loc[QUOTE_ASSET, 'Locked']
     total_portfolio_value_approx += usdt_bal
     logging.info(f"USDT Balance (Free+Locked): {usdt_bal:.2f}")
     balances_df.drop(QUOTE_ASSET, inplace=True, errors='ignore')

logging.info("Identifying non-USDT assets to flatten...")
for asset, row in balances_df.iterrows():
    free_balance = row['Free']
    if free_balance <= 0: continue # Can only sell free balance

    symbol = f"{asset}{QUOTE_ASSET}"
    ticker_info = all_tickers.get(symbol)
    if not ticker_info:
        logging.warning(f" - No USDT ticker found for {asset}. Cannot estimate value/flatten.")
        continue

    try:
        current_price = Decimal(ticker_info['lastPrice'])
        asset_value_usdt = free_balance * current_price
        total_portfolio_value_approx += asset_value_usdt # Add to total value estimate

        if asset_value_usdt >= MIN_VALUE_TO_SELL_USDT:
            assets_to_flatten.append({
                "asset": asset, "symbol": symbol, "balance": free_balance,
                "approx_value": asset_value_usdt, "current_price": current_price
            })
            logging.info(f" - WILL FLATTEN: {asset} (Balance: {free_balance:.8f}, Value: ~${asset_value_usdt:.2f})")
        else:
            logging.info(f" - IGNORING DUST: {asset} (Value: ~${asset_value_usdt:.2f})")
    except Exception as e:
        logging.warning(f" - Error processing {asset}: {e}")


logging.info(f"\nApproximate Total Portfolio Value: ${total_portfolio_value_approx:.2f}")
logging.info(f"Identified {len(assets_to_flatten)} assets to attempt flattening.")

# Store for next cell
identified_assets = assets_to_flatten

# End of Cell 2

2025-04-10 01:33:17,568 - INFO - --- Fetching Balances & Identifying Assets to Flatten ---
2025-04-10 01:33:17,639 - INFO - Fetched 11 non-zero balance entries.
2025-04-10 01:33:17,640 - INFO - Fetching all ticker prices...
2025-04-10 01:33:17,749 - INFO - Fetched 572 tickers.
2025-04-10 01:33:17,751 - INFO - USDT Balance (Free+Locked): 24.53
2025-04-10 01:33:17,752 - INFO - Identifying non-USDT assets to flatten...
2025-04-10 01:33:17,753 - INFO -  - IGNORING DUST: BTC (Value: ~$0.55)
2025-04-10 01:33:17,754 - INFO -  - IGNORING DUST: ETH (Value: ~$0.14)
2025-04-10 01:33:17,754 - INFO -  - WILL FLATTEN: XRP (Balance: 1.96400000, Value: ~$3.96)
2025-04-10 01:33:17,755 - INFO -  - IGNORING DUST: ADA (Value: ~$0.05)
2025-04-10 01:33:17,756 - INFO -  - IGNORING DUST: LINK (Value: ~$0.11)
2025-04-10 01:33:17,756 - INFO -  - IGNORING DUST: BUSD (Value: ~$0.00)
2025-04-10 01:33:17,757 - INFO -  - IGNORING DUST: WAVES (Value: ~$0.00)
2025-04-10 01:33:17,757 - INFO -  - WILL FLATTEN: USDC (Bal

In [3]:
# Cell 2: Fetch Balances & Identify Assets to Flatten

import pandas as pd
from decimal import Decimal
import logging

# --- Prerequisites ---
if 'client' not in locals() or client is None: raise RuntimeError("Client not initialized.")
if 'QUOTE_ASSET' not in locals(): locals()['QUOTE_ASSET'] = 'USDT' # Default if not set
if 'MIN_VALUE_TO_SELL_USDT' not in locals(): locals()['MIN_VALUE_TO_SELL_USDT'] = Decimal('1.00') # Default

logging.info("--- Fetching Balances & Identifying Assets to Flatten ---")

# --- Fetch Balances ---
portfolio_state = {"balances": pd.DataFrame()}
try:
    account_info = client.get_account()
    balances_raw = account_info.get('balances', [])
    processed = []
    for item in balances_raw:
        try: free = Decimal(item['free']); locked = Decimal(item['locked'])
        except: continue # Skip if conversion fails
        if free > 0 or locked > 0:
             processed.append({'Asset': item['asset'], 'Free': free, 'Locked': locked})
    if processed: portfolio_state["balances"] = pd.DataFrame(processed).set_index('Asset')
    logging.info(f"Fetched {len(portfolio_state['balances'])} non-zero balance entries.")
except Exception as e: raise SystemExit(f"Failed to fetch balances: {e}")

# --- Fetch Tickers for Valuation ---
all_tickers = {}
try:
    logging.info("Fetching all ticker prices..."); all_tickers = {t['symbol']: t for t in client.get_ticker()}
    logging.info(f"Fetched {len(all_tickers)} tickers.")
except Exception as e: logging.error(f"Failed to fetch tickers: {e}")

# --- Identify Assets ---
balances_df = portfolio_state.get('balances', pd.DataFrame()).copy()
identified_assets = [] # Renamed variable
total_portfolio_value_approx = Decimal('0')

if QUOTE_ASSET in balances_df.index:
     usdt_bal = balances_df.loc[QUOTE_ASSET, 'Free'] + balances_df.loc[QUOTE_ASSET, 'Locked']
     total_portfolio_value_approx += usdt_bal; logging.info(f"USDT Balance: {usdt_bal:.2f}")
     balances_df.drop(QUOTE_ASSET, inplace=True, errors='ignore')

logging.info("Identifying non-USDT assets to flatten...")
for asset, row in balances_df.iterrows():
    free_balance = row['Free']
    if free_balance <= 0: continue
    symbol = f"{asset}{QUOTE_ASSET}"; ticker_info = all_tickers.get(symbol)
    if not ticker_info: logging.warning(f" - No USDT ticker for {asset}. Skip."); continue
    try:
        current_price = Decimal(ticker_info['lastPrice']); asset_value_usdt = free_balance * current_price
        total_portfolio_value_approx += asset_value_usdt
        if asset_value_usdt >= MIN_VALUE_TO_SELL_USDT:
            identified_assets.append({"asset": asset, "symbol": symbol, "balance": free_balance, "approx_value": asset_value_usdt, "current_price": current_price})
            logging.info(f" - WILL FLATTEN: {asset} (Bal: {free_balance:.8f}, Val: ~${asset_value_usdt:.2f})")
        else: logging.info(f" - IGNORING DUST: {asset} (Val: ~${asset_value_usdt:.2f})")
    except Exception as e: logging.warning(f" - Error processing {asset}: {e}")

logging.info(f"\nApproximate Total Portfolio Value: ${total_portfolio_value_approx:.2f}")
logging.info(f"Identified {len(identified_assets)} assets to attempt flattening.")

# End of Cell 2

2025-04-10 01:33:17,769 - INFO - --- Fetching Balances & Identifying Assets to Flatten ---
2025-04-10 01:33:17,841 - INFO - Fetched 11 non-zero balance entries.
2025-04-10 01:33:17,842 - INFO - Fetching all ticker prices...
2025-04-10 01:33:17,961 - INFO - Fetched 572 tickers.
2025-04-10 01:33:17,962 - INFO - USDT Balance: 24.53
2025-04-10 01:33:17,963 - INFO - Identifying non-USDT assets to flatten...
2025-04-10 01:33:17,964 - INFO -  - IGNORING DUST: BTC (Val: ~$0.55)
2025-04-10 01:33:17,965 - INFO -  - IGNORING DUST: ETH (Val: ~$0.14)
2025-04-10 01:33:17,966 - INFO -  - WILL FLATTEN: XRP (Bal: 1.96400000, Val: ~$3.96)
2025-04-10 01:33:17,966 - INFO -  - IGNORING DUST: ADA (Val: ~$0.05)
2025-04-10 01:33:17,967 - INFO -  - IGNORING DUST: LINK (Val: ~$0.11)
2025-04-10 01:33:17,968 - INFO -  - IGNORING DUST: BUSD (Val: ~$0.00)
2025-04-10 01:33:17,968 - INFO -  - IGNORING DUST: WAVES (Val: ~$0.00)
2025-04-10 01:33:17,969 - INFO -  - WILL FLATTEN: USDC (Bal: 1.98800000, Val: ~$1.99)
2025-

In [4]:
# Cell 4: Place Flattening SELL Orders (Simplified - TypeError Fix v3)

import logging
from decimal import Decimal, ROUND_DOWN, ROUND_UP, InvalidOperation
import time
import math
import pandas as pd

# --- Prerequisites ---
if 'client' not in locals() or client is None: raise RuntimeError("Client not initialized.")
if 'identified_assets' not in locals():
     if 'identified_assets' in locals() and isinstance(identified_assets, list) and not identified_assets:
          logging.info("No assets identified in Cell 2 meeting criteria. Nothing to place.")
          placement_results = {"success": [], "failed": [], "skipped": []} # Ensure exists
     else: raise RuntimeError("Asset list ('identified_assets') not found. Run Cell 2 first.")
else:
    logging.info(f"--- Attempting to Place {len(identified_assets)} Flattening SELL Orders (Simplified) ---")

    # --- Config (Assume from Cell 1) ---
    QUOTE_ASSET = locals().get('QUOTE_ASSET', 'USDT')
    PRICING_STRATEGY = locals().get('PRICING_STRATEGY', 'BELOW_ASK')
    ORDER_BOOK_DEPTH = locals().get('ORDER_BOOK_DEPTH', 5)
    INTER_ORDER_DELAY_SECONDS = locals().get('INTER_ORDER_DELAY_SECONDS', 0.2)

    # --- Fetch Exchange Info ---
    symbol_info_lookup = {}
    try:
        if 'symbol_info_lookup' not in locals() or not symbol_info_lookup:
            logging.info("Fetching exchange info for filters..."); exchange_info = client.get_exchange_info(); all_symbols_info = exchange_info.get('symbols', []); symbol_info_lookup = {s['symbol']: s for s in all_symbols_info}
            logging.info("Fetched exchange info.")
    except Exception as e: raise SystemExit(f"Failed to get exchange info: {e}")

    # --- Placement Results Tracking ---
    placement_results = {"success": [], "failed": [], "skipped": []}
    assets_processed_count = 0

    # --- Helper Functions (Defined within this cell for self-containment) ---
    def adjust_price_up(price, tick_size):
        try:
            if not isinstance(price, Decimal) or not isinstance(tick_size, Decimal) or tick_size <= 0: return price # Return original if invalid
            ticks = price / tick_size; adjusted_ticks = ticks.to_integral_value(rounding=ROUND_UP); return adjusted_ticks * tick_size
        except Exception: return price # Return original on error
    def adjust_price(price, tick_size): # Adjust Down
        try:
            if not isinstance(price, Decimal): return None
            if not isinstance(tick_size, Decimal) or tick_size <= 0: return None
            adjusted_val = (price // tick_size) * tick_size
            return adjusted_val
        except Exception as e: logging.error(f"Error in adjust_price: {e}"); return None
    def adjust_qty(quantity, min_q, step_size):
        try:
            # Explicitly check for None and correct types
            if quantity is None or min_q is None or step_size is None:
                 logging.error("adjust_qty received None input.")
                 return Decimal('0')
            if not isinstance(quantity,Decimal) or not isinstance(min_q,Decimal) or not isinstance(step_size,Decimal):
                 logging.error(f"adjust_qty received non-Decimal: Q({type(quantity)}) MinQ({type(min_q)}) Step({type(step_size)})")
                 return Decimal('0')
            if step_size <= 0:
                logging.warning(f"adjust_qty received invalid step_size: {step_size}. Returning 0.")
                return Decimal('0')
            if quantity < min_q: return Decimal('0')
            # Perform calculation
            num_steps = math.floor((quantity - min_q) / step_size); adj_qty = min_q + (num_steps * step_size); return adj_qty
        except Exception as e: logging.error(f"Unexpected error in adjust_qty: {e}"); return Decimal('0')
    def format_decimal_for_api(value: Decimal, step_or_tick_size: Decimal) -> str:
        try:
            if not isinstance(value, Decimal) or not isinstance(step_or_tick_size, Decimal) or step_or_tick_size <= 0: return f"{value:.8f}"
            decimal_places = abs(step_or_tick_size.normalize().as_tuple().exponent); return f"{value:.{decimal_places}f}"
        except Exception: return f"{value:.8f}"
    # --- End Helper Functions ---

    # --- Processing Loop ---
    for asset_info in identified_assets:
        assets_processed_count += 1
        symbol = asset_info['symbol']; balance_to_sell = asset_info['balance']; current_price_estimate = asset_info['current_price']
        logging.info(f"\n[{assets_processed_count}/{len(identified_assets)}] Processing {symbol}...")

        # --- Get Filters & Minimum Check ---
        s_info = symbol_info_lookup.get(symbol)
        if not s_info: logging.warning(f"  Missing filters. Skip."); placement_results["skipped"].append({"symbol": symbol, "reason": "Missing filters"}); continue
        price_tick_size=None; min_qty=None; qty_step_size=None; min_notional=Decimal('0') # Init as None
        try: # Parse Filters
            price_tick_size_str = None; min_qty_str = None; qty_step_size_str = None; min_notional_str = '0'
            for f in s_info.get('filters',[]):
                if f['filterType']=='PRICE_FILTER': price_tick_size_str=f.get('tickSize')
                elif f['filterType']=='LOT_SIZE': min_qty_str=f.get('minQty'); qty_step_size_str=f.get('stepSize')
                elif f['filterType']=='NOTIONAL': min_notional_str=f.get('minNotional','0')
            # Convert after loop, checking if found
            if price_tick_size_str is None or min_qty_str is None or qty_step_size_str is None: raise ValueError("Essential filter(s) missing")
            price_tick_size=Decimal(price_tick_size_str); min_qty=Decimal(min_qty_str); qty_step_size=Decimal(qty_step_size_str); min_notional=max(Decimal('0'), Decimal(min_notional_str))
            if price_tick_size<=0 or qty_step_size<=0 or min_qty<0: raise ValueError("Invalid filter values") # Allow min_qty=0? No, must be > 0 to trade
            if min_qty == 0: raise ValueError("minQty filter is zero")

            min_value_from_qty = min_qty * current_price_estimate
            opportunistic_min_target_usdt = min_value_from_qty * Decimal('1.01')
            effective_required = max(opportunistic_min_target_usdt, min_notional)
            # logging.info(f"  Filters: Tick={price_tick_size}, MinQty={min_qty}, Step={qty_step_size}, RepMinNot={min_notional}")
            # logging.info(f"  Checks: MinQtyVal=~{min_value_from_qty:.4f}, EffReq=~{effective_required:.4f}")
            if balance_to_sell * current_price_estimate < effective_required:
                 logging.warning(f"  Value (~${balance_to_sell * current_price_estimate:.2f}) < EffMin ${effective_required:.2f}. Skip.")
                 placement_results["skipped"].append({"symbol": symbol, "reason": "Balance value below effective min"})
                 continue
        except (ValueError, TypeError, InvalidOperation) as filter_e: logging.error(f"  Filter error: {filter_e}. Skip."); placement_results["skipped"].append({"symbol": symbol, "reason": f"Filter error: {filter_e}"}); continue
        except Exception as filter_e: logging.error(f"  Unexpected Filter error: {filter_e}. Skip."); placement_results["skipped"].append({"symbol": symbol, "reason": f"Filter error: {filter_e}"}); continue


        # --- Get Order Book & Calculate Target Price ---
        target_price_adjusted = None
        try: # Get Depth & Calc Price
            # ... (Get depth, best_bid, best_ask) ...
            depth = client.get_order_book(symbol=symbol, limit=ORDER_BOOK_DEPTH)
            best_bid = Decimal(depth['bids'][0][0]) if depth.get('bids') else None; best_ask = Decimal(depth['asks'][0][0]) if depth.get('asks') else None
            if not best_bid or not best_ask: raise ValueError("Empty book side")
            price_precision = abs(price_tick_size.normalize().as_tuple().exponent)
            # logging.info(f"  Book: Bid={best_bid:.{price_precision}f}, Ask={best_ask:.{price_precision}f}")
            target_price_raw = None
            if PRICING_STRATEGY == 'BID': target_price_raw = best_bid
            elif PRICING_STRATEGY == 'BELOW_ASK': target_price_raw = best_ask - price_tick_size
            elif PRICING_STRATEGY == 'MID': target_price_raw = (best_bid + best_ask) / Decimal('2')
            else: target_price_raw = best_bid
            if target_price_raw is None: raise ValueError("Raw price calc failed")

            target_price_adjusted = adjust_price(target_price_raw, price_tick_size) # Adjust DOWN
            if target_price_adjusted is None: raise ValueError("Adjust price failed (returned None)") # Check for None
            if target_price_adjusted <= 0: raise ValueError(f"Adjusted price <=0 ({target_price_adjusted})") # Check <= 0

        except Exception as book_price_e: logging.error(f"  Book/Price error: {book_price_e}. Skip."); placement_results["skipped"].append({"symbol": symbol, "reason": f"Book/Price error: {book_price_e}"}); continue

        # --- Adjust Quantity ---
        # Pre-check inputs before calling adjust_qty
        if balance_to_sell is None or min_qty is None or qty_step_size is None:
             logging.error(f"  Cannot adjust quantity due to None inputs. Skipping.")
             placement_results["skipped"].append({"symbol": symbol, "reason": "None input to adjust_qty"})
             continue
        quantity_adjusted = adjust_qty(balance_to_sell, min_qty, qty_step_size)

        # Check result of adjust_qty
        if quantity_adjusted is None: # Defensive check
             logging.error(f"  adjust_qty returned None unexpectedly. Skipping.")
             placement_results["skipped"].append({"symbol": symbol, "reason": "adjust_qty returned None"})
             continue
        if quantity_adjusted <= 0:
             logging.warning(f"  Adjusted quantity zero or negative ({quantity_adjusted}). Skipping.")
             placement_results["skipped"].append({"symbol": symbol, "reason": f"Adjusted quantity <=0 ({quantity_adjusted})"})
             continue

        # --- Final Notional Check ---
        final_notional = target_price_adjusted * quantity_adjusted
        if final_notional < opportunistic_min_target_usdt:
             logging.warning(f"  Final notional ${final_notional:.2f} < OppMinTgt ${opportunistic_min_target_usdt:.2f}. Skip.")
             placement_results["skipped"].append({"symbol": symbol, "reason": f"Final notional below opp min"})
             continue

        # --- Format & Place Order ---
        try: # Format
            price_str = format_decimal_for_api(target_price_adjusted, price_tick_size)
            quantity_str = format_decimal_for_api(quantity_adjusted, qty_step_size)
            logging.info(f"  Attempting: SELL {quantity_str} {symbol} @ {price_str} (Value ~${final_notional:.2f})")
            # Place Order (No orderRespType)
            sell_order_info = client.create_order(symbol=symbol, side=client.SIDE_SELL, type=client.ORDER_TYPE_LIMIT, timeInForce=client.TIME_IN_FORCE_GTC, quantity=quantity_str, price=price_str)
            logging.info(f"    SUCCESS: Flatten order placed (ACK). OrderId: {sell_order_info.get('orderId')}")
            placement_results["success"].append({ "symbol": symbol, "order_info": sell_order_info })
        # ... (Exception handling for placement) ...
        except (BinanceAPIException, BinanceOrderException) as e:
             if e.code == -1013 and "NOTIONAL" in e.message: logging.error(f"    FAILED (MIN_NOTIONAL): Value ${final_notional:.2f}. Code={e.code}")
             else: logging.error(f"    FAILED: Code={e.code}, Msg={e.message}")
             placement_results["failed"].append({ "symbol": symbol, "error_code": e.code, "error_message": e.message })
        except Exception as e:
             logging.error(f"    FAILED: Unexpected Error: {e}")
             placement_results["failed"].append({ "symbol": symbol, "error_code": None, "error_message": f"Unexpected: {e}"})

        time.sleep(INTER_ORDER_DELAY_SECONDS)

# --- Summary ---
# ... (Summary printing remains the same) ...
print("\n--- Position Flattening Placement Summary ---")
print(f"Assets identified: {len(identified_assets)}")
print(f"Orders skipped pre-placement: {len(placement_results['skipped'])}")
if placement_results["skipped"]:
     for skip in placement_results["skipped"]: print(f"  - Skipped {skip['symbol']}: {skip['reason']}")
print(f"Successful placements: {len(placement_results['success'])}")
if placement_results["success"]:
     for success in placement_results["success"]: print(f"  - Success {success['symbol']}: OrderId {success['order_info'].get('orderId')}")
print(f"Failed placements: {len(placement_results['failed'])}")
if placement_results["failed"]:
     for failure in placement_results["failed"]: print(f"  - Failed {failure['symbol']}: Code={failure.get('error_code', 'N/A')}, Msg='{failure.get('error_message', 'N/A')}'")


# End of Cell 4 (Flatten Utility - Simplified Place Logic - TypeError Fix v3)

2025-04-10 01:33:18,029 - INFO - --- Attempting to Place 3 Flattening SELL Orders (Simplified) ---
2025-04-10 01:33:18,030 - INFO - Fetching exchange info for filters...
2025-04-10 01:33:18,207 - INFO - Fetched exchange info.
2025-04-10 01:33:18,207 - INFO - 
[1/3] Processing XRPUSDT...
2025-04-10 01:33:18,275 - WARNING -   Final notional $2.02 < OppMinTgt $2.04. Skip.
2025-04-10 01:33:18,275 - INFO - 
[2/3] Processing USDCUSDT...
2025-04-10 01:33:18,343 - WARNING -   Final notional $1.00 < OppMinTgt $1.01. Skip.
2025-04-10 01:33:18,343 - INFO - 
[3/3] Processing HBARUSDT...
2025-04-10 01:33:18,412 - INFO -   Attempting: SELL 12 HBARUSDT @ 0.16884 (Value ~$2.03)
2025-04-10 01:33:18,480 - INFO -     SUCCESS: Flatten order placed (ACK). OrderId: 95888791

--- Position Flattening Placement Summary ---
Assets identified: 3
Orders skipped pre-placement: 2
  - Skipped XRPUSDT: Final notional below opp min
  - Skipped USDCUSDT: Final notional below opp min
Successful placements: 1
  - Success